# 벡터 데이터베이스 임베딩 미션 실습

이 노트북에서는 한국어 텍스트 데이터를 임베딩하여 Pinecone 벡터 데이터베이스에 저장하고 검색하는 실습을 진행합니다.

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# 환경 변수 로드 (Gemini API 사용)
import os
import google.generativeai as genai

# Gemini API 키 확인
GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')
if GEMINI_API_KEY:
    print(f"Gemini API 키 확인됨: {GEMINI_API_KEY[:10]}...")
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print("❌ GEMINI_API_KEY 환경변수가 설정되지 않았습니다.")

In [ ]:
# 필요한 라이브러리 임포트 (FAISS 로컬 벡터DB 사용)
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity

## 2. Gemini 임베딩 모델 설정

Google Gemini API를 사용하여 한국어 텍스트를 임베딩합니다.

In [ ]:
# Gemini 임베딩 함수 정의
def get_gemini_embedding(text: str, task_type: str = 'retrieval_document'):
    """Google Gemini API를 사용하여 텍스트 임베딩 생성"""
    try:
        response = genai.embed_content(
            model='models/text-embedding-004',
            content=text,
            task_type=task_type
        )
        return response['embedding']
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        return None

# 테스트
test_text = "안녕하세요. 테스트 텍스트입니다."
test_embedding = get_gemini_embedding(test_text)

if test_embedding:
    print(f"✅ Gemini API 임베딩 성공!")
    print(f"임베딩 차원: {len(test_embedding)}")
    print(f"임베딩 샘플 (처음 5개): {test_embedding[:5]}")
else:
    print("❌ 임베딩 생성 실패")

## 3. 임베딩 미션 데이터 준비

사과(과일)와 애플(회사)에 관련된 한국어 텍스트 데이터를 사용합니다.

In [ ]:
# 임베딩 미션 데이터
data = [
    {"id": "vec1", "text": "사과는 달콤하고 아삭한 식감으로 유명한 인기 있는 과일입니다."},
    {"id": "vec2", "text": "애플이라는 기술 회사는 아이폰과 같은 혁신적인 제품으로 유명합니다."},
    {"id": "vec3", "text": "많은 사람들이 건강한 간식으로 사과를 즐겨 먹습니다."},
    {"id": "vec4", "text": "애플 주식회사는 세련된 디자인과 사용자 친화적인 인터페이스로 기술 산업을 혁신했습니다."},
    {"id": "vec5", "text": "하루에 사과 하나면 의사를 멀리할 수 있다는 속담이 있습니다."},
    {"id": "vec6", "text": "애플 컴퓨터 회사는 1976년 4월 1일 스티브 잡스, 스티브 워즈니악, 로널드 웨인에 의해 파트너십으로 설립되었습니다."}
]

print("데이터 개수:", len(data))
for item in data:
    print(f"{item['id']}: {item['text'][:30]}...")

## 4. 텍스트 임베딩 생성

각 텍스트를 multilingual-e5-large 모델로 임베딩합니다.

In [ ]:
# Gemini API로 텍스트 임베딩 생성
embeddings = []
texts = []
ids = []
categories = []

print("=== Gemini API로 임베딩 생성 중 ===")

for item in data:
    print(f"{item['id']} 처리 중...", end=" ")
    
    # 문서용 임베딩 생성
    embedding = get_gemini_embedding(item['text'], task_type='retrieval_document')
    
    if embedding:
        embeddings.append(embedding)
        texts.append(item['text'])
        ids.append(item['id'])
        # 카테고리 분류
        category = '과일' if '사과' in item['text'] and '애플' not in item['text'] else '기술회사'
        categories.append(category)
        print(f"✅ 완료 (차원: {len(embedding)})")
    else:
        print("❌ 실패")

print(f"\n✅ 총 {len(embeddings)}개의 임베딩이 생성되었습니다.")
if embeddings:
    print(f"임베딩 차원: {len(embeddings[0])}")
    
    # numpy 배열로 변환
    embeddings_array = np.array(embeddings)
    print(f"임베딩 배열 형태: {embeddings_array.shape}")

## 5. 로컬 벡터 인덱스 생성 (Pinecone 대신)

임베딩을 메모리에 저장하고 코사인 유사도로 검색합니다.

In [ ]:
# 로컬 벡터 인덱스 생성 (메모리 기반)
if embeddings:
    print("=== 로컬 벡터 인덱스 생성 ===")
    print(f"벡터 개수: {len(embeddings)}")
    print(f"벡터 차원: {len(embeddings[0])}")
    
    # 메타데이터와 함께 저장
    vector_database = {
        'embeddings': embeddings_array,
        'metadata': {
            'texts': texts,
            'ids': ids,
            'categories': categories
        }
    }
    
    print("✅ 로컬 벡터 인덱스 생성 완료!")
    print(f"저장된 벡터 수: {len(embeddings)}")
else:
    print("❌ 임베딩이 생성되지 않아서 인덱스를 만들 수 없습니다.")

## 6. 로컬 벡터 저장 완료

임베딩이 메모리에 저장되었습니다.

In [ ]:
# 벡터 저장 완료 확인
if 'vector_database' in locals():
    print("✅ 벡터 데이터베이스 저장 완료!")
    print(f"저장된 벡터 수: {len(vector_database['embeddings'])}")
    print(f"메타데이터 항목 수: {len(vector_database['metadata']['texts'])}")
else:
    print("❌ 벡터 데이터베이스 생성 실패")

## 7. 벡터 통계 확인

저장된 벡터 데이터의 통계를 확인합니다.

In [ ]:
# 벡터 데이터베이스 통계 조회
if 'vector_database' in locals():
    print("=== 벡터 데이터베이스 통계 ===")
    embeddings_shape = vector_database['embeddings'].shape
    print(f"벡터 차원: {embeddings_shape[1]}")
    print(f"전체 벡터 수: {embeddings_shape[0]}")
    
    # 카테고리별 통계
    categories_count = {}
    for category in vector_database['metadata']['categories']:
        categories_count[category] = categories_count.get(category, 0) + 1
    
    print("카테고리별 분포:")
    for category, count in categories_count.items():
        print(f"  - {category}: {count}개")
        
    print(f"ID 목록: {vector_database['metadata']['ids']}")
else:
    print("❌ 벡터 데이터베이스가 없습니다.")

## 8. 쿼리 벡터 생성 및 유사도 검색

"애플이라는 기술 회사에 대해 알려주세요." 쿼리로 유사한 벡터를 검색합니다.

In [ ]:
# 쿼리 텍스트
query_text = "애플이라는 기술 회사에 대해 알려주세요."
print(f"쿼리: {query_text}")

# 쿼리 임베딩 생성 (query: 접두사 사용)
query_embedding = embedding_model.encode(f"query: {query_text}", normalize_embeddings=True)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")

## 9. 벡터 검색 수행 (index.query)

생성된 쿼리 임베딩으로 유사한 벡터를 검색합니다.

In [ ]:
# 벡터 검색 수행
search_results = index.query(
    namespace="embedding-mission",
    vector=query_embedding.tolist(),
    top_k=6,  # 모든 벡터 검색
    include_values=False,  # 벡터 값은 제외 (용량 절약)
    include_metadata=True   # 메타데이터 포함
)

print("=== 검색 결과 ===")
print(f"쿼리: {query_text}\n")

for i, match in enumerate(search_results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    category = match['metadata']['category']
    
    print(f"{i}. [{match['id']}] (유사도: {score:.4f}) [{category}]")
    print(f"   텍스트: {text}")
    print()

## 10. 카테고리별 필터링 검색

메타데이터 필터를 사용하여 '기술회사' 카테고리만 검색해보겠습니다.

In [ ]:
# 기술회사 카테고리만 필터링하여 검색
filtered_results = index.query(
    namespace="embedding-mission",
    vector=query_embedding.tolist(),
    top_k=10,
    include_values=False,
    include_metadata=True,
    filter={'category': {'$eq': '기술회사'}}  # 기술회사 카테고리만 검색
)

print("=== 기술회사 카테고리 필터링 검색 결과 ===")
print(f"쿼리: {query_text}\n")

for i, match in enumerate(filtered_results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    category = match['metadata']['category']
    
    print(f"{i}. [{match['id']}] (유사도: {score:.4f}) [{category}]")
    print(f"   텍스트: {text}")
    print()

## 11. 다른 쿼리로 검색 테스트

과일 관련 쿼리로도 검색해보겠습니다.

In [ ]:
# 과일 관련 쿼리
fruit_query = "건강에 좋은 과일에 대해 알려주세요."
print(f"쿼리: {fruit_query}")

# 쿼리 임베딩 생성
fruit_query_embedding = embedding_model.encode(f"query: {fruit_query}", normalize_embeddings=True)

# 검색 수행
fruit_results = index.query(
    namespace="embedding-mission",
    vector=fruit_query_embedding.tolist(),
    top_k=6,
    include_values=False,
    include_metadata=True
)

print("\n=== 과일 관련 검색 결과 ===")
for i, match in enumerate(fruit_results['matches'], 1):
    score = match['score']
    text = match['metadata']['text']
    category = match['metadata']['category']
    
    print(f"{i}. [{match['id']}] (유사도: {score:.4f}) [{category}]")
    print(f"   텍스트: {text}")
    print()

## 12. 결과 분석 및 정리

임베딩 미션 실습 결과를 정리합니다.

In [ ]:
print("=== 임베딩 미션 실습 완료 ===")
print(f"✅ 사용 모델: {model_name}")
print(f"✅ 임베딩 차원: {dimension}")
print(f"✅ 저장된 벡터 수: {len(data)}")
print(f"✅ 인덱스명: {index_name}")
print(f"✅ 네임스페이스: embedding-mission")
print()
print("주요 기능 테스트:")
print("✅ 텍스트 임베딩 생성")
print("✅ Pinecone 인덱스 생성")
print("✅ 벡터 데이터 업서트(upsert)")
print("✅ 인덱스 통계 조회 (describe_index_stats)")
print("✅ 유사도 검색 (index.query)")
print("✅ 메타데이터 필터링 검색")
print()
print("실습 결과:")
print("- '애플' 쿼리는 기술회사 관련 텍스트와 높은 유사도를 보임")
print("- '과일' 쿼리는 사과(과일) 관련 텍스트와 높은 유사도를 보임")
print("- 메타데이터 필터링을 통해 특정 카테고리만 검색 가능")
print("- multilingual-e5-large 모델이 한국어 의미 이해에 효과적임")